In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Security/filename.csv')


In [ ]:
data.head(2)

,Mturk_id,base_rate,phase,trial,email_type,email_id,user_action1,user_action2,user_action3,time,score,cum_score,Sender,Subject,Email,Email_type,Cluster_no
0,A100Y89FZO4J0B,75,1,1,ham,209,0,70,4,2019-06-03 13:03:00,1,1,discount@eddiebauer.com,STARTS TODAY! 40% Off Everything!,<br>Save on everything you need for your next ...,Ham,4
1,A100Y89FZO4J0B,75,2,1,phishing,159,0,60,3,2019-06-03 13:14:00,0,9,newoffers@jpmorgan.bank.com,New Mortgage Offer!,<p>Dear Loyal Customer:</p><p>&nbsp;</p><p>JP ...,Phishing,6


In [ ]:
data.drop(columns=['Mturk_id', 'phase', 'trial', 'email_type', 'email_id', 'user_action2', 'user_action3', 'time', 'score', 'cum_score', 'Email_type', 'Cluster_no'], inplace=True)

In [ ]:
print(data.columns)

Index(['base_rate', 'user_action1', 'Sender', 'Subject', 'Email'], dtype='object')


In [ ]:
features = data.drop(columns=['user_action1'])
target = data['user_action1']

categorical_cols = features.select_dtypes(include=['object', 'category']).columns
numeric_cols = features.select_dtypes(include=['int64', 'float64']).columns

column_transformer = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ]
)

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:
pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  Index(['base_rate'], dtype='object')),
                                                 ('cat', OneHotEncoder(),
                                                  Index(['Sender', 'Subject', 'Email'], dtype='object'))])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [ ]:
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.84      0.80      1866
           1       0.82      0.73      0.77      1804

    accuracy                           0.79      3670
   macro avg       0.79      0.79      0.79      3670
weighted avg       0.79      0.79      0.79      3670

